In [1]:
pip install neuralcoref

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install scikit-multilearn

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pickle
import re
import xml.etree.ElementTree as ET

import neuralcoref
import nltk
import pandas as pd
import spacy
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MultiLabelBinarizer
from skmultilearn.problem_transform import LabelPowerset

C:\Users\Sabby\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: spacy.morphology.Morphology size changed, may indicate binary incompatibility. Expected 104 from C header, got 112 from PyObject
  return f(*args, **kwds)
C:\Users\Sabby\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: spacy.vocab.Vocab size changed, may indicate binary incompatibility. Expected 96 from C header, got 112 from PyObject
  return f(*args, **kwds)
C:\Users\Sabby\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: spacy.tokens.span.Span size changed, may indicate binary incompatibility. Expected 72 from C header, got 80 from PyObject
  return f(*args, **kwds)
100%|██████████████████████████████████████████████████████████████████| 40155833/40155833 [00:12<00:00, 3252520.08B/s]


In [5]:
# # Uncomment this
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Sabby\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Sabby\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [6]:
tree = ET.parse('C:/Users/Sabby/Downloads/NLP Assignment 3/Restaurants_Train.xml')
root = tree.getroot()

Parsing Xml File 

In [7]:
labeled_reviews = []

for sentence in root.findall("sentence"):
    entry = {}
    aterms = []
    aspects = []
    if sentence.find("aspectTerms"):
        for aterm in sentence.find("aspectTerms").findall("aspectTerm"):
            aterms.append(aterm.get("term"))
    if sentence.find("aspectCategories"):
        for aspect in sentence.find("aspectCategories").findall("aspectCategory"):
            aspects.append(aspect.get("category"))
    entry["text"], entry["terms"], entry["aspects"]= sentence[0].text, aterms, aspects
    labeled_reviews.append(entry)

labeled_df = pd.DataFrame(labeled_reviews)


print(f'There are {len(labeled_reviews)} reviews in this training set')

There are 3041 reviews in this training set


In [9]:
# Save annotated reviews

labeled_df.to_pickle("C:/Users/Sabby/Downloads/NLP Assignment 3/annotated_reviews_df.pkl")
labeled_df.head()

,text,terms,aspects
0,But the staff was so horrible to us.,[staff],[service]
1,"To be completely fair, the only redeeming fact...",[food],"[food, anecdotes/miscellaneous]"
2,"The food is uniformly exceptional, with a very...","[food, kitchen, menu]",[food]
3,Where Gabriela personaly greets you and recomm...,[],[service]
4,"For those that go once and don't enjoy it, all...",[],[anecdotes/miscellaneous]


In [11]:
pip install en

  Created wheel for en: filename=en-0.0.1-cp37-none-any.whl size=1156 sha256=2276b37920db48ab847950e0a3124754050f3de4f6b838ed8d2219359fb757d7
  Stored in directory: C:\Users\Sabby\AppData\Local\pip\Cache\wheels\00\43\90\056879336c6fad1a3c6b467a91f507232904a939224871653c
Successfully built en
Note: you may need to restart the kernel to use updated packages.


In [10]:
nlp = spacy.load('en')

neuralcoref.add_to_pipe(nlp)

# Define function for replacing pronouns using neuralcoref
def replace_pronouns(text):
    doc = nlp(text)
    return doc._.coref_resolved

OSError: [E050] Can't find model 'en'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [9]:
# read annotated reviews DataFrame

annotated_reviews_df = pd.read_pickle("/content/sample_data/annotated_reviews_df.pkl")
annotated_reviews_df.head()

,text,terms,aspects
0,But the staff was so horrible to us.,[staff],[service]
1,"To be completely fair, the only redeeming fact...",[food],"[food, anecdotes/miscellaneous]"
2,"The food is uniformly exceptional, with a very...","[food, kitchen, menu]",[food]
3,Where Gabriela personaly greets you and recomm...,[],[service]
4,"For those that go once and don't enjoy it, all...",[],[anecdotes/miscellaneous]


In [ ]:
# Create a new column with fixed co-referencing

annotated_reviews_df["fixed_review"] = annotated_reviews_df["text"].map(lambda x: replace_pronouns(x))

annotated_reviews_df.head()